In [ ]:
!pip install opencv-python-headless


In [ ]:
"""
convolutional neural network (CNN) for Handwritten Digit Recognition
"""

# Larger CNN for the MNIST Dataset

# Import necessary libraries
import numpy  # For numerical operations
from tensorflow import keras
from keras.datasets import mnist  # For loading the MNIST dataset
from keras.models import Sequential  # For building a sequential neural network model
from keras.layers import Dense  # For creating fully connected (dense) layers
from keras.layers import Dropout  # For adding dropout regularization
from keras.layers import Flatten  # For flattening 2D matrices into 1D vectors
from keras.layers import Conv2D  # For adding 2D convolutional layers
from keras.layers import MaxPooling2D  # For adding max pooling layers
from tensorflow.keras.utils import to_categorical  # for utility functions, including one-hot encoding



In [ ]:
#  For image prediction
import cv2
import matplotlib.pyplot as plt
from keras.models import load_model
from google.colab import drive  # Only needed in Colab

In [ ]:


# Fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# Load the MNIST dataset
# The dataset consists of images of handwritten digits (0-9) and their corresponding labels
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshape the data to [samples][width][height] [channels]for CNN input
# Each image is 28x28 pixels, and we add a single channel (1) for grayscale
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

# Normalize inputs from 0-255 to 0-1
# Pixel values are scaled to the range [0, 1] to improve model training performance
X_train = X_train / 255
X_test = X_test / 255

# One-hot encode the output labels
# The labels are integers (0-9), converted to one-hot encoded vectors for multi-class classification
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
num_classes = y_test.shape[1]  # Number of output classes (10 digits in total)

# Define the larger CNN model as a function
def larger_model():
    # Create a sequential model
    model = Sequential()

    # Add a 2D convolutional layer with 30 filters, 5x5 kernel, and ReLU activation
    # Input shape for the first layer must match the reshaped image data (1 channel, 28x28)
    model.add(Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'))

    # Add a max pooling layer with a 2x2 pooling window
    # This reduces the spatial dimensions by half, helping to downsample the image
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Add a second convolutional layer with 15 filters and 3x3 kernel with ReLU activation
    model.add(Conv2D(15, (3, 3), activation='relu'))

    # Add another max pooling layer with a 2x2 pooling window
    # Further reduces spatial dimensions
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Add a dropout layer with 20% dropout rate to prevent overfitting
    model.add(Dropout(0.2))

    # Flatten the feature maps into a 1D vector for input into the fully connected layers
    model.add(Flatten())

    # Add a fully connected (dense) layer with 128 neurons and ReLU activation
    model.add(Dense(128, activation='relu'))

    # Add another fully connected layer with 50 neurons and ReLU activation
    model.add(Dense(50, activation='relu'))

    # Add the output layer with 'num_classes' neurons and softmax activation
    # Softmax activation is used to output probabilities for each class (0-9)
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    # Categorical crossentropy is the loss function for multi-class classification
    # Adam optimizer is used for training, and accuracy is tracked as a performance metric
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Build the model by calling the larger_model function
model = larger_model()

# Train the model

# Fit the model on the training data
# We specify validation data (X_test, y_test) for evaluating the model's accuracy on unseen data during training
# Epochs = 10 means the model will iterate over the data 10 times
# Batch size = 200 means the model will update weights after every 200 samples
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)

# Evaluate model

# Final evaluation of the model on the test data
scores = model.evaluate(X_test, y_test, verbose=0)

# Print the error rate of the model on the test data
# Baseline error is calculated as (100 - accuracy), where accuracy is given as a percentage
print("Large CNN Error: %.2f%%" % (100 - scores[1] * 100))


In [ ]:
#Save model
model.save('/content/drive/MyDrive/digit_model.h5')  # Save model to Google Drive
print("✅ Model saved to Google Drive.")


In [ ]:
 #  Predict digit from image
# Mount Google Drive (if using Colab)
drive.mount('/content/drive')


In [ ]:

# Path to the digit image in your Drive
image_path = '/content/sample_data/number.jpeg'



In [ ]:



# Load image in grayscale
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

if img is None:
    print(" Error: Image not found. Check the path.")
else:
    # Resize to 28x28
    img = cv2.resize(img, (28, 28))

    # Invert colors (white background, black digit)
    img = cv2.bitwise_not(img)

    # Show the image
    plt.imshow(img, cmap='gray')
    plt.title("Input Image")
    plt.axis('off')
    plt.show()

    # Normalize and reshape
    img = img.astype('float32') / 255.0
    img = img.reshape(1, 28, 28, 1)

    # Load the model (again, to show how it works after saving)
    model = load_model('/content/drive/MyDrive/digit_model.h5')

    # Predict
    prediction = model.predict(img)
    predicted_digit = numpy.argmax(prediction)

    print(f" Predicted digit is: {predicted_digit}")